In [1]:
# takes 3m or 6s per file
# download tables from https://www.bridgepowerratings.com/prhome.htm
# use acbl api to convert Name into player_id.
# create dicts helpful to converting Name->player_id and player_id->Name.
# deal with duplicate names and missing player_ids.

# previous steps:
# none

# next steps:
# merge bpr data in acbl_pair_summaries.ipynb and useful places.

# todo:
# implement teams (4+ players) similar to list used in pair_ids.
# should keys in acbl_player_name_to_id_dict use casefold() or not?

In [2]:
import pandas as pd
import pathlib
from collections import defaultdict
import mlBridgeLib

In [3]:
# override pandas display options
mlBridgeLib.pd_options_display()

In [4]:
rootPath = pathlib.Path('e:/bridge/data')
acblPath = rootPath.joinpath('acbl')
acblPath.mkdir(parents=True,exist_ok=True)
bprPath = rootPath.joinpath('bpr')
bprPath.mkdir(parents=True,exist_ok=True)

In [5]:
# takes 4s per file
# create dataframes from bridgepowerratings.com webpage tables.
# experimenting with flavor='bs4'. seems to be faster for this webpage.
pages = ['WEBPAGES/allmps.htm','WEBPAGES/allpr.htm','WEBPAGES/improv6.htm','WEBPAGES/improv12.htm','WEBPAGES/improv24.htm','WEBPAGES/underate.htm','WEBPAGES/urpairs.htm']
pages += [f'WEBPAGES/DISTRICT/d{d}rank.htm' for d in range(1,26)] + ['WEBPAGES/DISTRICT/d99rank.htm'] # include districts 1-25, 99
dfsd = {}
for p in pages:
    url = f'https://www.bridgepowerratings.com/{p}'
    print(f'Reading: {url}')
    # read_html() retuns a list of dataframes. The dataframes are populated from tables embedded in the html.
    dfs = pd.read_html(url, flavor='bs4', encoding='UTF8')
    print(f'Table count: {len(dfs)}')
    dfs = [v for v in dfs if len(set(v.iloc[0])) != len(v.iloc[0])] # kludgy way to detect and bypass any tables which are composed of shortcuts to names.
    row1_cols = dfs[0].iloc[0]
    ncols = len(row1_cols)
    nnames = len(set(row1_cols))
    col_names = row1_cols[:nnames]
    #print(row1_cols,ncols,nnames,col_names)
    df = pd.concat([pd.DataFrame(df.iloc[1:,ncol:ncol+nnames].values,columns=col_names) for df in dfs for ncol in range(0,ncols,nnames)],ignore_index=True).dropna()
    # use url to create a filename
    n = pathlib.Path(url).stem
    dfsd[n] = df
    display(df)

Reading: https://www.bridgepowerratings.com/WEBPAGES/allmps.htm
Table count: 53


,Rank,MP's,Unit,Name
0,1,92862.58,128,"Meckstroth, Jeff"
1,2,87910.58,176,"Passell, Mike"
2,3,78168.46,128,"Rodwell, Eric"
3,4,76525.13,174,"Wold, Eddie"
4,5,71817.34,210,"Lair, Mark"
5,6,62350.16,538,"Itabashi, Mark"
6,7,51506.72,373,"Hampson, Geoff"
7,8,50419.44,176,"Compton, Chris"
8,9,47550.39,373,"Levin, Robert"
9,10,46419.44,373,"Cheek, Curtis"


Reading: https://www.bridgepowerratings.com/WEBPAGES/allpr.htm
Table count: 51


,Rank,PR,Games,Unit,Name
0,1,72.09,137,108,"Grossack, Zachary"
1,2,71.27,60,174,"Wold, Eddie"
2,3,71.14,86,373,"Shi, Sylvia"
3,4,71.02,201,538,"Itabashi, Mark"
4,5,71.01,164,128,"Cappelletti, Mike"
5,6,70.76,47,128,"Meckstroth, Jeff"
6,7,70.28,131,151,"Pollack, Frederic"
7,8,69.67,77,155,"Nathan, Marc"
8,9,69.59,69,128,"Stamatov, Jerry"
9,10,69.37,198,155,"Rotaru, Iulian"


Reading: https://www.bridgepowerratings.com/WEBPAGES/improv6.htm
Table count: 52


...

,Rank,PR,Games,Unit,Name
0,1,69.67,77,155,"Nathan, Marc"
1,2,69.37,198,155,"Rotaru, Iulian"
2,3,69.34,153,155,"Nistor, Radu"
3,4,69.13,119,155,"Grue, Joe"
4,5,68.45,150,155,"Radin, Judi"
5,6,68.40,34,155,"Hurd, John"
6,7,67.97,280,155,"Tokay, Mustafa Cem"
7,8,67.17,181,155,"Wooldridge, Joel"
8,9,67.07,110,155,"Jansma, Jan"
9,10,66.85,224,155,"Gurvich, David"


Reading: https://www.bridgepowerratings.com/WEBPAGES/DISTRICT/d25rank.htm
Table count: 7


,Rank,PR,Games,Unit,Name
0,1,72.09,137,108,"Grossack, Zachary"
1,2,68.62,50,108,"Grossack, Adam"
2,3,66.87,114,150,"Arvedon, Lloyd"
3,5,65.25,37,126,"Stiefel, John"
4,6,64.90,137,108,"Aquino, Mark"
5,7,64.79,50,108,"Montague, Brenda"
6,8,64.41,96,108,"Luba, Harrison"
7,9,64.19,46,108,"Watson, Alan"
8,10,64.10,41,113,"Hanerfeld, Larry"
9,11,63.97,83,126,"Doub, Douglas"


Reading: https://www.bridgepowerratings.com/WEBPAGES/DISTRICT/d99rank.htm
Table count: 2


,Rank,PR,Games,Unit,Name
0,1,66.76,153,999,"Isporski, Vladislav"
1,2,64.38,22,999,"Sigurjonsson, Julius"
2,3,62.90,131,999,"Friedlander, Ehud"
3,4,62.52,35,999,"Zadroga, Jan"
4,5,60.71,18,999,"Lee, Kang-Won"
5,6,59.90,47,999,"Greenberg, Shmuel"
6,7,55.47,38,999,"Deva, Yogendra"
7,8,55.26,57,999,"SHIELS, Brian"
8,9,53.63,93,999,"Lapage, Robert"
9,10,53.54,101,999,"Bruchansky, Peggy"


In [6]:
# Reading acbl_player_name_dict which has been created from sql db which was created by club data download.
# This file is the starting point for creating the larger player_id_name_matches_dict which is supplemented with api calls.
if False:
    acbl_player_id_to_name_dict_file = acblPath.joinpath('acbl_player_name_dict.pkl')
    acbl_player_id_to_name_dict = pd.read_pickle(acbl_player_id_to_name_dict_file)
    # Create a list of player_ids for each name. Many players share a common name but each has a unique player_id.
    acbl_player_name_to_id_dict = defaultdict(list)
    for k,v in acbl_player_id_to_name_dict.items():
        acbl_player_name_to_id_dict[v].append(k)

In [7]:
# ignoring acbl_player_id_to_name_dict and using the more complete player_id_name_matches_dict.pkl instead.
if True:
    player_id_name_matches_dict_file = acblPath.joinpath('player_id_name_matches_dict.pkl')
    acbl_player_name_to_id_dict = pd.read_pickle(player_id_name_matches_dict_file) # note name change!
    for k,v in acbl_player_name_to_id_dict.items():
        assert type(k) is str, k
        if type(v) is list:
            v = acbl_player_name_to_id_dict[k] = [str(vv) for vv in v] # todo: temp: force ints in list to be strs.
            assert all(type(vv) is str for vv in v), v
        else:
            assert type(v) is str and v[0] != '[', v

In [8]:
# for player dataframes, create new column containing player_id
# for pair dataframes, create new columns of lists containing pair_ids, Names
for k,df in dfsd.items():
    print(df.columns)
    if 'PairR' in df:
        df['Names'] = df['Name'].map(lambda r: r.split(' - ')) # if 'Name' is missing, re-download dfs
        df['pair_ids'] = df['Names'].map(lambda r: [acbl_player_name_to_id_dict[n] for n in r]) # pair of ids and names
        df.drop('Name',axis='columns',inplace=True)
    else:
        df['player_id'] = df['Name'].map(lambda n: acbl_player_name_to_id_dict[n] if n in acbl_player_name_to_id_dict else []) # scaler id and name
    display(k,df)

Index(['Rank', 'MP's', 'Unit', 'Name'], dtype='object', name=0)


'allmps'

,Rank,MP's,Unit,Name,player_id
0,1,92862.58,128,"Meckstroth, Jeff",[4580699]
1,2,87910.58,176,"Passell, Mike",[1622382]
2,3,78168.46,128,"Rodwell, Eric",[5482658]
3,4,76525.13,174,"Wold, Eddie",[3443949]
4,5,71817.34,210,"Lair, Mark",[5830400]
5,6,62350.16,538,"Itabashi, Mark",[6811434]
6,7,51506.72,373,"Hampson, Geoff",[9984488]
7,8,50419.44,176,"Compton, Chris",[8007713]
8,9,47550.39,373,"Levin, Robert",[7279515]
9,10,46419.44,373,"Cheek, Curtis",[2931060]


Index(['Rank', 'PR', 'Games', 'Unit', 'Name'], dtype='object', name=0)


'allpr'

...

'd25rank'

,Rank,PR,Games,Unit,Name,player_id
0,1,72.09,137,108,"Grossack, Zachary",[2250896]
1,2,68.62,50,108,"Grossack, Adam",[8808155]
2,3,66.87,114,150,"Arvedon, Lloyd",[4592433]
3,5,65.25,37,126,"Stiefel, John",[4396715]
4,6,64.90,137,108,"Aquino, Mark",[3627667]
5,7,64.79,50,108,"Montague, Brenda",[9995145]
6,8,64.41,96,108,"Luba, Harrison",[5476992]
7,9,64.19,46,108,"Watson, Alan",[6167373]
8,10,64.10,41,113,"Hanerfeld, Larry",[1303643]
9,11,63.97,83,126,"Doub, Douglas",[6981321]


Index(['Rank', 'PR', 'Games', 'Unit', 'Name'], dtype='object', name=0)


'd99rank'

,Rank,PR,Games,Unit,Name,player_id
0,1,66.76,153,999,"Isporski, Vladislav",[4972619]
1,2,64.38,22,999,"Sigurjonsson, Julius",[5162823]
2,3,62.90,131,999,"Friedlander, Ehud",[4074009]
3,4,62.52,35,999,"Zadroga, Jan",[4410165]
4,5,60.71,18,999,"Lee, Kang-Won",[]
5,6,59.90,47,999,"Greenberg, Shmuel",[2268248]
6,7,55.47,38,999,"Deva, Yogendra","[tmp:6d128041-b217-4a1d-9d4e-af509c91f778, 8447020, tmp:598cb48f-d796-4515-8270-9e85846ee17d, tm..."
7,8,55.26,57,999,"SHIELS, Brian",[]
8,9,53.63,93,999,"Lapage, Robert",[]
9,10,53.54,101,999,"Bruchansky, Peggy",[]


In [9]:
# create a list of names containing missing player_id or pair_ids.
l = [] # todo: change to list which only allows uniques. setlist?
for k,df in dfsd.items():
    if 'player_id' in df:
        df[df['player_id'].map(len).eq(0)]['Name'].map(lambda n: l.append(n.casefold()))
    elif 'pair_ids' in df:
        df.apply(lambda r: [l.append(name.casefold()) for lpid,name in zip(r['pair_ids'],r['Names']) if len(lpid) == 0],axis='columns')
    else:
        assert False
missing_player_ids = set(l) - acbl_player_name_to_id_dict.keys() # only need to call apis for names not in dict
# todo: should dict contain casefold() or not? Need consistency.
print([n for n in missing_player_ids if n in acbl_player_name_to_id_dict])
assert all([n not in acbl_player_name_to_id_dict for n in missing_player_ids])
len(l),len(missing_player_ids),sorted(missing_player_ids)

[]


(2231,
 99,
 ["a'hearn, beverly",
  'abramson-borson, janet',
  'bacon-rothchild, debora',
  'bennett-perry, patricia',
  'benoit, marie-france',
  'berton, jr, jack',
  'bickel-scribner, susan',
  'bigel-casher, rita',
  'brown, jr, nelson',
  'burgess-strauss, marie',
  'cannamela, jr, r. arthu',
  'coburn, donald',
  'cooper-baker, rebecca',
  "d'amato, ellen",
  "d'amico, james",
  "d'litzenberger, richard",
  "d'souza, lino",
  'day, jo-ann',
  "del'monte, ishmael",
  "dell'osso, john",
  'dickson, jo-beth',
  'el-raheb, ibrahem',
  'emmett, mary-lou',
  'fill in',
  'fu, tsun-sen',
  'fung-a-fat, shirley',
  'goggin, m. gregory',
  'goodrich-baird, sarah',
  'gruskay, robert',
  'hamilton-diesel, stepha',
  'harmon-hanna, e',
  'ho, sun-o',
  'housman jr, w.glover',
  'hsiau, yao-hui',
  'hunt, samuel',
  'hyde, judith',
  'jean-jacques, palau',
  'kay-wolff, judy',
  'keegan, james',
  'ketchum, jr, william',
  'khaitan, viswanath',
  "l'ecuyer, nicolas",
  'lacombe-ingham, sher

In [10]:
# read file containing acbl api bearer secret.
bearer_file = pathlib.Path('../../../BridgePowerRatings/acbl_api_authorization_bearer_secret.txt')
with open(bearer_file,'rb') as f:
    bearer = f.read()

In [11]:
# takes 60s per 40 names
# call acbl api to retrieve acbl_number given first_name, last_name.
import requests
import urllib
import time
headers = {'accept':'application/json', 'Authorization':bearer[len('Authorization: '):]}
unmatched_names = []
for nn,n in enumerate(missing_player_ids):
    if n in acbl_player_name_to_id_dict and len(acbl_player_name_to_id_dict[n]):
        #print('found:',n,acbl_player_name_to_id_dict[n])
        continue
    split = n.split(', ')
    if len(split) != 2:
        print('wrong number of commas:',n)
        unmatched_names.append(n)
        continue
    lastname = split[0].split(' ')[0]
    if not lastname.isalnum() or '-' in lastname or '+' in lastname or '.' in lastname or ',' in lastname:
        print('invalid characters in lastname:',lastname,n)
        unmatched_names.append(n)
        continue
    firstname = split[1].split(' ')[0]
    if not firstname.isalnum() or '-' in firstname or '+' in firstname or '.' in firstname or ',' in firstname:
        print('invalid characters in firstname:',firstname,n)
        unmatched_names.append(n)
        continue
    url = 'https://api.acbl.org/v1/player_query'
    query = {'first_name':firstname,'last_name':lastname}
    params = urllib.parse.urlencode(query)
    time.sleep(0.5) # throttle api calling
    print(url,params)
    response = requests.get(url, params=params, headers=headers)
    assert response.status_code == 200, (n, url, params, response.status_code)
    json_response = response.json()
    founds = []
    for data in json_response['data']:
        acbl_number = data['acbl_number']
        assert type(acbl_number) is int
        if firstname.casefold() == data['first_name'].casefold() and lastname.casefold() == data['last_name'].casefold():
            founds += [(n,acbl_number,data['city'])]
    if len(founds) == 1:
        print('found one exact match:',n)
    else:
        founds = [(data['last_name']+', '+data['first_name'],data['acbl_number'],data['city']) for data in json_response['data']]
        unmatched_names.append(n)
    print(nn,len(missing_player_ids),n,founds)
    acbl_player_name_to_id_dict[n] = [n[1] for n in founds]

invalid characters in lastname: o'connor o'connor wade, nora
invalid characters in lastname: el-raheb el-raheb, ibrahem
invalid characters in firstname: e.r. mckinney, e.r. 'bud'
invalid characters in lastname: o'donnell o'donnell, kevin
invalid characters in lastname: bacon-rothchild bacon-rothchild, debora
wrong number of commas: berton, jr, jack
invalid characters in lastname: wall-gilletz wall-gilletz, susan
https://api.acbl.org/v1/player_query first_name=viswanath&last_name=khaitan
found one exact match: khaitan, viswanath
7 99 khaitan, viswanath [('khaitan, viswanath', 5514746, 'Wellesley')]
invalid characters in lastname: weisz-margules weisz-margules, adam
invalid characters in firstname: tsun-sen fu, tsun-sen
invalid characters in lastname: o'rourke o'rourke, lou ann
invalid characters in firstname: w.glover housman jr, w.glover
invalid characters in lastname: lacombe-ingham lacombe-ingham, sherry
invalid characters in lastname: o'connor o'connor, timothy
invalid characters in

In [12]:
# use same dict for both player_id->name lookup and name->[player_id] lookup
# key can be either a name or a player_id. value can be either a name or a list of player_ids.
d = {}
name_differences = []
for k,v in acbl_player_name_to_id_dict.items():
    assert type(k) is str, k
    if type(v) is list:
        # v is a list of player_ids. make sure every player_id can retieve its name.
        for pid in v:
            if type(pid) is int:
                pid = str(pid)
            assert type(pid) is str, pid
            if pid in acbl_player_name_to_id_dict:
                if acbl_player_name_to_id_dict[pid] != k:
                    # different names are using the same player_id. what to do?
                    name_differences.append([pid,[k,acbl_player_name_to_id_dict[pid]]])
            else:
                if pid in d:
                    assert d[pid] == k, [d[pid],k]
                else:
                    d[pid] = k
    else:
        assert type(k) is str, k
print('new player_ids to add to dict:',len(d))
acbl_player_name_to_id_dict.update(d)

new player_ids to add to dict: 3


In [13]:
# todo: resolve name differences by making acbl api call to convert player_id to name? Better to keep as is?
print('Name differences:')
for pid,names in name_differences:
    print(pid, names)

Name differences:
1083023 ['Moore, Tom', 'Moore, Loren']
7402872 ['Brantley, Theresa', 'Brantley, Theresa Ann']
6497594 ['Langland, Richard', 'lang, richard']
7749635 ['Walker, Jim', 'Walker, James']
1889443 ['Depner, Garnet', 'De Merchant, Garnet']
7006837 ['Panayotatos, Daniele-Favre', 'Panayotatos, Daniele']
6697151 ['Hendricks, Bob', 'Hendricks, Robert']
1448854 ['Black, Tom', 'Black, Thomas']
3105741 ['Jones, Hal', 'Jones Jr, Hal']
3629538 ['Rosenstock, Norm', 'Rosenstock, Norman']
2376318 ['Scott, Candy', 'Scott, Candace']
1718967 ['Scott, Patsy', 'scott, patricia']
3174603 ['Elson, Jane', 'Elson, Marjorie J']
6094481 ['Kirk, Pat', 'Kirk, Patricia']
5244781 ['Hirshon, Ken', 'Hirshon, Kenneth']
8114978 ['Laslie, MaryPat', 'Laslie, Mary Pat']
3429598 ['Dennis, Leona', 'De Grandpre, Leona']
2050153 ['Andrews, Mike', 'Andrews, Michael']
9999310 ['Sorenson, Eric', 'Sorenson, Steven']
9746129 ['Harker, Bill', 'Harker, William']
4989988 ['Templeton, Max', 'templeton, maxie']
4187458 ['Y

2704390 ['Kelly, Rick', 'Kelly, Richard']
7070977 ['Wood, Karen', 'Wood, K']
6762840 ['Dietrick, Bob', 'Dietrick, Robert']
9582452 ['Archer, Wendell', 'Archer, Wendel']
3134539 ['Vernick, Marty', 'vernick, martin']
5726344 ['Hoffman, Roberta', 'hoffman jr, robert']
3806944 ['Roraback, Charlie', 'roraback, charles']
5884691 ['Zoltak, Margaret Mary', 'Zoltak, Margaret']
7334699 ['Carr, Tom', 'carr, thomas']
6083234 ['Overholt, Rich', 'Overholt, Richard']
7395426 ['Burman, Ronnie', 'Burman, Ritambhar']
5723639 ['Baechi, Joe', 'bacchi, joseph']
6434134 ['Siskind, Judy', 'Siskind, Judith']
2066270 ['Steele, Douglas', 'St John, Douglas']
5998840 ['WANG, YAN', 'Wang, Yan']
1349929 ['Snider, Kathy', 'snider, kathleen']
5470196 ['Porter, Steve', 'porter, stephen']
6621201 ['KAPLAN, ELAINE', 'Kaplan, Elaine']
8221480 ['Chakraborty, Chuck', 'Chakraborty, Ranajit']
5773245 ['Bowlan, James R', 'bowlan, james']
3820416 ['Burns, Dan', 'burns, daniel']
1105868 ['Owen, Jim', 'owen, james']
6656404 ['Dr

9402381 ['Fraser, Robert W', 'fraser, robert']
4777859 ['Briggs, Nancy C.', 'Briggs, Nancy']
3133966 ['Malet, Phil', 'malet, philip']
9512829 ['Dentai, Andrew', 'De Leon, Andrew']
7302851 ['Dalton, Elizabeth', 'Dalton, E']
6048277 ['Leifert, Shirlee', 'Leifert, Shirley']
7757506 ['Simpson, Marvin', 'Simpson, Marty']
9020454 ['Simmons, Lou', 'Simmons, Louis']
9648836 ['Abramson, Larry', 'abramson, laurence']
7672659 ['Patel, Kishorshandra', 'patel, kishor']
5955793 ['Harding, Rene', 'Harding, Irene']
4633539 ['Condon, Gene', 'Condon, Eugene']
8702799 ['Dennerline, Robert', 'denner, robert']
2979772 ['Kirkpatrick, Patty', 'Kirk, Patricia']
7337493 ['Fohey, Mike', 'fohey, michael']
9235574 ['Ries, Andy', 'ries, andrew']
7314752 ['Rossman, Lawrence', 'rossman, ross l']
5986850 ['Chiodo, Fran', 'chiodo, frances']
6971989 ['Prashad, Jai', 'prashad, jaikaran']
8810214 ['Bell, Steve', 'bell, stephen']
5957109 ['Lins, Rich', 'lins, richard']
7326068 ['Dubester, Judy', 'dubester, judith']
675739

In [14]:
# takes 25s
# write out dicts of matched and unmatched name/player_ids in pkl and excel formats.
player_id_name_matches_file = acblPath.joinpath('player_id_name_matches_dict.pkl')
pd.to_pickle(acbl_player_name_to_id_dict, player_id_name_matches_file) # create dict file
player_id_name_matches_df = pd.DataFrame({'Name':acbl_player_name_to_id_dict.keys(),'player_id':acbl_player_name_to_id_dict.values()})
player_id_name_matches_df.to_excel(player_id_name_matches_file.with_suffix('.xlsx')) # create excel file of matches
player_id_name_unmatches_file = acblPath.joinpath('player_id_name_unmatches_dict.pkl')
player_id_name_unmatches_df = pd.DataFrame({'Name':unmatched_names,'player_id':[None]*len(unmatched_names)})
player_id_name_unmatches_df.to_pickle(player_id_name_unmatches_file) # don't bother creating an unmatched df?
player_id_name_unmatches_df.to_excel(player_id_name_unmatches_file.with_suffix('.xlsx')) # create excel file of unmatches
len(missing_player_ids),len(player_id_name_matches_df),len(player_id_name_unmatches_df),len(acbl_player_name_to_id_dict)

(99, 286189, 89, 286189)

In [15]:
# takes 10s
# for each df, assign any missing player_ids using newly updated dict.
# Some player_ids will be missing because acbl API retrival is flawed e.g. api rejects names containing ' .,+-' etc.
for k,df in dfsd.items():
    print(k,df.columns,k in df.columns)
    if 'player_id' in df:
        df['player_id'] = df.apply(lambda r: r['player_id'] if r['player_id'] else acbl_player_name_to_id_dict[r['Name']] if r['Name'] in acbl_player_name_to_id_dict else [],axis='columns')
        display(k,len(df),'Players missing player_id:',df[df['player_id'].map(lambda pid: len(pid) == 0)])
        df_file = bprPath.joinpath(k+'.pkl')
        df.to_pickle(df_file) # create df file
        df.to_excel(df_file.with_suffix('.xlsx')) # create excel file
    elif 'pair_ids' in df:
        df['pair_ids'] = df.apply(lambda r: [[str(pid) for pid in lpid] if lpid else acbl_player_name_to_id_dict[name] for lpid,name in zip(r['pair_ids'],r['Names'])],axis='columns')
        display(k,len(df),'Pairs missing a player_id:',df[df['pair_ids'].map(lambda llpid: any([len(lpid)==0 for lpid in llpid]))])
        df_file = bprPath.joinpath(k+'.pkl')
        df.to_pickle(df_file) # create df file
        df.to_excel(df_file.with_suffix('.xlsx')) # create excel file
    else:
        assert False
    display(df.head())

allmps Index(['Rank', 'MP's', 'Unit', 'Name', 'player_id'], dtype='object', name=0) False


'allmps'

5000

'Players missing player_id:'

,Rank,MP's,Unit,Name,player_id
198,199,17664.60,354,"O'Rourke, Lou Ann",[]
255,256,15961.75,446,"Ho, Sun-O",[]
470,471,12557.40,151,"L'Ecuyer, Nicolas",[]
526,527,12166.49,373,"Del'Monte, Ishmael",[]
846,847,10054.72,999,"Jean-Jacques, Palau",[]
1331,1332,8231.90,218,"Lo, Ai-Tai",[]
1531,1532,7653.56,166,"D'Souza, Lino",[]
1725,1726,7160.18,373,"Kay-Wolff, Judy",[]
2613,2614,5708.77,253,"Cannamela, Jr, R. Arthu",[]
2838,2839,5453.64,207,"O'Donnell, Kevin",[]


...

'd99rank'

12

'Players missing player_id:'

,Rank,PR,Games,Unit,Name,player_id
4,5,60.71,18,999,"Lee, Kang-Won",[]
7,8,55.26,57,999,"SHIELS, Brian",[]
8,9,53.63,93,999,"Lapage, Robert",[]
9,10,53.54,101,999,"Bruchansky, Peggy",[]
11,12,50.69,88,999,"Traub, Dan",[]


,Rank,PR,Games,Unit,Name,player_id
0,1,66.76,153,999,"Isporski, Vladislav",[4972619]
1,2,64.38,22,999,"Sigurjonsson, Julius",[5162823]
2,3,62.90,131,999,"Friedlander, Ehud",[4074009]
3,4,62.52,35,999,"Zadroga, Jan",[4410165]
4,5,60.71,18,999,"Lee, Kang-Won",[]


In [16]:
# do some validation
for k,df in dfsd.items():
    print(k,df.columns)
    if 'player_id' in df:
        for l in df['player_id'].values:
            assert type(l) is list, [type(l),l]
            for v in l:
                assert type(v) is str, v
    elif 'pair_ids' in df:
        for ll in df['pair_ids'].values:
            assert type(ll) is list, [type(ll),ll]
            for l in ll:
                assert type(l) is list, [type(l),l]
                for v in l:
                    assert type(v) is str, v
    else:
        assert False

allmps Index(['Rank', 'MP's', 'Unit', 'Name', 'player_id'], dtype='object', name=0)
allpr Index(['Rank', 'PR', 'Games', 'Unit', 'Name', 'player_id'], dtype='object', name=0)
improv6 Index(['Rank', 'last/PR', 'old/PR', 'PR/inc', 'Unit', 'Name', 'player_id'], dtype='object', name=0)
improv12 Index(['Rank', 'last/PR', 'old/PR', 'PR/inc', 'Unit', 'Name', 'player_id'], dtype='object', name=0)
improv24 Index(['Rank', 'last/PR', 'old/PR', 'PR/inc', 'Unit', 'Name', 'player_id'], dtype='object', name=0)
underate Index(['Rank', 'UR', 'PR/MP's', 'MP's', 'PR', 'Unit', 'Name', 'player_id'], dtype='object', name=0)
urpairs Index(['Rank', 'UR', 'PR/MP's', 'MP's', 'PairR', 'Unit', 'Names', 'pair_ids'], dtype='object', name=0)
d1rank Index(['Rank', 'PR', 'Games', 'Unit', 'Name', 'player_id'], dtype='object', name=0)
d2rank Index(['Rank', 'PR', 'Games', 'Unit', 'Name', 'player_id'], dtype='object', name=0)
d3rank Index(['Rank', 'PR', 'Games', 'Unit', 'Name', 'player_id'], dtype='object', name=0)
d4rank 